In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Prediction algorithms (supervised learning)
1. Linear Regression for predicted population count for each winner (multivariate linear regression)


1. Logistic (get beta values, or log odds ratio and look at predict proba)
2. KNN
3. Gradient Boosting Classifier (Random Forest)
Don't forget confusion matrix!!

4. We need state results (and electoral college votes) to see if they match for sum of county results.

Can we get updated demographic information to predict the 2020 elections?

5. Can we scrape twitter data and look at popularity? Maybe use NLP/NMF to look at latent features for the different tweets? Too ambitious....
6. Hate crime statistics? Starter code to download from github is in data_clean script




In [2]:
# Import data
pd.set_option("display.max_columns", 40)
data = pd.read_csv('data/cleaned_data2.csv')
data.head()

,Unnamed: 0,state,county,fips,white_pct,black_pct,hispanic_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,rural_pct,trump16_pct,clinton16_pct,otherpres16_pct,romney12_pct,obama12_pct,otherpres12_pct,cvap_pct12,metro,rural,urban_metroadj,urban_not_metroadj
0,0,Alabama,Autauga,1001,75.683482,18.370906,2.572254,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,42.002162,72.766588,23.769671,3.463741,72.618252,26.587832,0.793916,58.815434,1,0,0,0
1,1,Alabama,Baldwin,1003,83.178788,9.225603,4.366698,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,42.279099,76.545712,19.385601,4.068687,77.358269,21.589444,1.052286,56.228504,1,0,0,0
2,2,Alabama,Barbour,1005,45.885624,47.888329,4.309762,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,67.789635,52.096666,46.527844,1.375490,48.223130,51.368494,0.408376,56.485890,0,0,1,0
3,3,Alabama,Bibb,1007,74.765196,21.212121,2.223994,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,68.352607,76.403220,21.249575,2.347205,72.826603,26.152019,1.021378,47.868107,1,0,0,0
4,4,Alabama,Blount,1009,87.657701,1.557951,8.727298,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,89.951502,89.334844,8.425825,2.239331,86.465884,12.371907,1.162209,56.577893,1,0,0,0


In [3]:
data.drop(columns=['Unnamed: 0', 'state', 'county', 'fips'], inplace=True)

In [4]:
# Split into metro, rural, urban 
metro = data[data['metro'] == 1]
rural = data[data['rural'] == 1]
urban_metroadj = data[data['urban_metroadj'] == 1]
urban_not_metroadj = data[data['urban_not_metroadj'] == 1]

In [ ]:
def train_test_split_rep(df):

    # Pop ys
    trump = df.pop('trump16_pct').values()
    clinton = df.pop('clinton16_pct').values()
    other = df.pop('otherpres16_pct').values()
    X = df.values()
        
    # Make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, trump)
    
    

In [8]:
# Extract our labels
trump = data.pop('trump16_pct')
clinton = data.pop('clinton16_pct')
other = data.pop('otherpres16_pct')

In [24]:
# Make our X and y and do 75%/25% split
X = data.values
y = trump.values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [25]:
X_train.shape, X_test.shape

((2333, 20), (778, 20))

## Test toy model before using Kfold

In [26]:
# Standardize. Fit model using only train data, but transform on all
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)


In [27]:
model = LinearRegression().fit(X_train, y_train)
y_hat = model.predict(X_test)

In [28]:
model.coef_

array([ 1.84502049e+00, -7.10549353e-01, -7.57517003e-01, -3.64191543e-01,
        1.35411122e-01, -4.85126024e-01,  3.18462826e-01,  6.74557186e-01,
       -5.93298946e-01,  5.05368347e-01,  3.86227030e+00,  3.97109204e-01,
       -3.96598584e+13, -3.93380982e+13, -3.73940322e+12, -6.44517933e-01,
        4.03480252e+12,  3.31648217e+12,  3.61646431e+12,  3.11532516e+12])

In [35]:
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)/y_hat.mean()

0.04927601898740639

In [ ]:
# Use kFold 
kf = KFold(n_splits=5)